## Парсинг данных из графика отпусков 

Берём данные из онлан инструмента https://app.vacationplanner.ru/ в которой ранее зарегистрировали сотрудников, поскольку искали готовое удобное решение с планирование отпусков. 

In [22]:
import requests
from bs4 import BeautifulSoup
url = "https://app.vacationplanner.ru/auth/process"
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
} 
data = {
    'login': '....',
    'password': '...',
    'code': '',
    'auth_mode': 'passwd'
}

session = requests.Session() # создаём сессию, которая будет работать на протяжении всего сеанса запросов
session.headers.update(headers)
response = session.post(url, data=data)

#### Создание html страницы с графиком отпусков сотрудников

In [24]:
response1 = session.get('https://app.vacationplanner.ru/schedule/show/year/2023')
soup = BeautifulSoup(response1.text, 'html.parser')

#### Создание html страницы со списком сотрудников по департаментам

In [122]:
response2 = session.get('https://app.vacationplanner.ru/workers')
soup1 = BeautifulSoup(response2.text, 'html.parser')

* Достанем список сотрудников в датафрейм

In [132]:
import pandas as pd
table = soup1.find('table', class_='table app-table-hover mb-1 text-left')
rows = table.find_all('tr')
employees = []
for i, row in enumerate(rows):
    if i==0: # пропускаем первую строку
        continue
    cells = row.find_all('td')
    name = cells[0].find('strong').text
    position = cells[0].find('i').text
    employee_id = cells[1].find('small').text
    department = cells[2].find('a').text if cells[2].find('a') else None
    employees.append([name, position, employee_id, department])
employees = pd.DataFrame(employees, columns=['Name', 'Position', 'Employee_id', 'Geo_'])
employees

,Name,Position,Employee_id,Geo_
0,Авдеев Иван Михайлович,Специалист 2-ой категории,ТН-002,Нск
1,Амренов Максат Ханатович,Специалист 2-ой категории,ТН-014,Мск
2,Васильев Роман Вячеславович,Специалист 2-ой категории,ТН-011,Мск
3,Ведяков Александр Евгеньевич,Специалист 2-ой категории,ТН-003,Мск
4,Галиев Ильнур Фанузович,Ведущий инженер-эколог,ТН-006,Мск
5,Ириняков Денис Сергеевич,Специалист 2 категории,,None
6,Ириняков Денис Сергеевич,Специалист 2-ой категории,ТН-012,Нск
7,Кадеркаев Денис Андреевич,инженер-эколог 3 категории,ТН-019,Мск
8,Калинин Станислав Константинович,Специалист 2-ой категории,ТН-007,Нск
9,Колесникова Элла Андреевна,Руководитель отдела логистики ЭМ,,Отдел логистики


5 строка здесь дублируется, поэтому удалим её

In [133]:
employees = employees.drop(employees.index[5])
employees

,Name,Position,Employee_id,Geo_
0,Авдеев Иван Михайлович,Специалист 2-ой категории,ТН-002,Нск
1,Амренов Максат Ханатович,Специалист 2-ой категории,ТН-014,Мск
2,Васильев Роман Вячеславович,Специалист 2-ой категории,ТН-011,Мск
3,Ведяков Александр Евгеньевич,Специалист 2-ой категории,ТН-003,Мск
4,Галиев Ильнур Фанузович,Ведущий инженер-эколог,ТН-006,Мск
6,Ириняков Денис Сергеевич,Специалист 2-ой категории,ТН-012,Нск
7,Кадеркаев Денис Андреевич,инженер-эколог 3 категории,ТН-019,Мск
8,Калинин Станислав Константинович,Специалист 2-ой категории,ТН-007,Нск
9,Колесникова Элла Андреевна,Руководитель отдела логистики ЭМ,,Отдел логистики
10,Колмаков Максим Алексеевич,Специалист 2-ой категории,ТН-009,Мск


* Теперь достанем данные по графику отпусков сотрудников

In [134]:
table = soup.find('table', class_='table table-hover')
rows = table.find_all('tr')
data = []
for row in rows:
    date_tag = row.find('td').find('small').find('i').find('nobr')
    date = date_tag.text
    start, end = date.split(" — ")
    name_tag = row.find_all('td')[1].find('small')
    name = name_tag.text
    data.append([name, start, end])
df = pd.DataFrame(data, columns=['Name', 'Start Date', 'End Date'])
print(df)

                                Name Start Date End Date
0          Корепанов Дмитрий Юрьевич     10 июл   23 июл
1             Авдеев Иван Михайлович     31 июл   13 авг
2        Сергеев Александр Сергеевич     12 июн   26 июн
3            Галиев Ильнур Фанузович     13 июн   18 июн
4       Ведяков Александр Евгеньевич     26 июн    9 июл
5   Калинин Станислав Константинович     24 июл    6 авг
6   Калинин Станислав Константинович     27 фев    5 мар
7            Галиев Ильнур Фанузович      7 авг   20 авг
8             Авдеев Иван Михайлович      6 фев   12 фев
9           Ириняков Денис Сергеевич      6 фев   19 фев
10             Торн Андрей Яковлевич     27 фев    5 мар
11          Ириняков Денис Сергеевич      8 мар   15 мар
12       Сергеев Александр Сергеевич     13 мар   19 мар
13            Самарин Иван Сергеевич     24 апр   30 апр
14      Ведяков Александр Евгеньевич     28 апр    5 мая
15            Самарин Иван Сергеевич     19 июн    2 июл
16       Васильев Роман Вячесла

Не нашел удобного метода преобразования росийских дат в даты в pandas. Поэтому пришлось написать большой костыль

In [135]:
df['Start Date']=df['Start Date'].replace(to_replace =' дек', value = '/12/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' янв', value = '/01/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' фев', value = '/02/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' мар', value = '/03/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' апр', value = '/04/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' мая', value = '/05/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' сен', value = '/09/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' окт', value = '/10/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' ноя', value = '/11/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' авг', value = '/08/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' июн', value = '/06/', regex = True)
df['Start Date']=df['Start Date'].replace(to_replace =' июл', value = '/07/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' дек', value = '/12/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' янв', value = '/01/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' фев', value = '/02/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' мар', value = '/03/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' апр', value = '/04/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' мая', value = '/05/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' сен', value = '/09/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' окт', value = '/10/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' ноя', value = '/11/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' авг', value = '/08/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' июн', value = '/06/', regex = True)
df['End Date']=df['End Date'].replace(to_replace =' июл', value = '/07/', regex = True)

In [136]:
df['Start Date'] = df['Start Date'].apply(lambda x: x + '2023')
df['End Date'] = df['End Date'].apply(lambda x: x + '2023')

Теперь даты можно превратить непосрдественно в даты и рассчитать продолжительность отпуска

In [143]:
df["Start Date"] = pd.to_datetime(df["Start Date"], format="%d/%m/%Y")
df["End Date"] = pd.to_datetime(df["End Date"], format="%d/%m/%Y")
df["Vacation Length"] = df["End Date"] - df["Start Date"]
df["Vacation Length"] = df["Vacation Length"].dt.days
df["Vacation Length"] = df["Vacation Length"]+1
df

,Name,Start Date,End Date,Vacation Length
0,Корепанов Дмитрий Юрьевич,2023-07-10,2023-07-23,14
1,Авдеев Иван Михайлович,2023-07-31,2023-08-13,14
2,Сергеев Александр Сергеевич,2023-06-12,2023-06-26,15
3,Галиев Ильнур Фанузович,2023-06-13,2023-06-18,6
4,Ведяков Александр Евгеньевич,2023-06-26,2023-07-09,14
5,Калинин Станислав Константинович,2023-07-24,2023-08-06,14
6,Калинин Станислав Константинович,2023-02-27,2023-03-05,7
7,Галиев Ильнур Фанузович,2023-08-07,2023-08-20,14
8,Авдеев Иван Михайлович,2023-02-06,2023-02-12,7
9,Ириняков Денис Сергеевич,2023-02-06,2023-02-19,14


### Ура! у меня получился первый небесполезный парсер. Далее м сделаем дашборд для визуализации графика отпусков.